In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import plotly.plotly as py
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()

from sklearn.cluster import KMeans

from features.feature_ts import *
from observations.obs_processing import *

## Retrieve Profiles

In [ ]:
year = 2000
unit = 'A'
directory = 'H'

In [ ]:
p = dailyProfiles(year, unit, directory)

## Explore Profiles

In [ ]:
p[1010:1100]

In [ ]:
p[-365:].T.iplot(kind='scatter', filename='cufflinks/cf-simple-line')

In [ ]:
p.describe()

In [ ]:
print('number of daily profiles: ' + str(p.shape[0]))
p.describe().loc['mean'].iplot(kind='scatter', filename='cufflinks/cf-simple-line')

## Cluster Profiles

In [ ]:
p_prep = p.dropna()
kmeans = KMeans(10, random_state=0).fit(p_prep)

### High Level Evaluation

In [ ]:
c_centres = pd.DataFrame(kmeans.cluster_centers_)
c_centres.T.iplot(kind='scatter', title='Plot of kmeans cluster centres', xTitle='hour of day', yTitle='mean hourly current (A)', filename='cufflinks/cf-simple-line')

In [ ]:
len(kmeans.labels_)

In [ ]:
pc = p_prep.assign(cluster=kmeans.labels_)

In [ ]:
c = pc.reset_index(drop=True)
c_mean = c.groupby('cluster').mean()
c_count = c.groupby('cluster').count() # frequency of cluster assignments

In [ ]:
#RMSE between cluster centre values and cluster mean values
rmse = np.sqrt((c_centres - c_mean)**2).mean(axis=1)
print(rmse)
rmse.sum()